In [ ]:
!pip3 install opencv-python
!pip3 install numpy
!pip3 install cvzone
!pip3 install mediapipe
!pip3 install protobuf==3.20.1
!pip3 install openvino

In [ ]:
import cv2
import numpy
import openvino
from cvzone.SelfiSegmentationModule import SelfiSegmentation

In [ ]:
# camera_location = 0 
camera_location = "vid.mp4"
cam = cv2.VideoCapture(camera_location)
vw  = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
vh = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
body_outline_source = cv2.imread("body_outline.png",cv2.IMREAD_GRAYSCALE)
body_outline_ratio = body_outline_source.shape[1]/body_outline_source.shape[0]
body_outline_source = cv2.resize(
    body_outline_source,
    (int(body_outline_ratio*vh),vh),
    cv2.INTER_AREA
)

In [ ]:
video_width_center = vw//2
body_width_center =  body_outline_source.shape[1]//2

In [ ]:
body_outline_mask = numpy.ones((vh,vw,3),dtype=numpy.uint8)
for y in range(0,body_outline_source.shape[0],1):
    for x in range(0,body_outline_source.shape[1],1):
        if(body_outline_source[y,x]!=255):
            body_outline_mask[y,video_width_center + (x - body_width_center),0] = 0
            body_outline_mask[y,video_width_center + (x - body_width_center),1] = 0
            body_outline_mask[y,video_width_center + (x - body_width_center),2] = 0
body_outline_mask = cv2.resize(body_outline_mask,(512,512),cv2.INTER_NEAREST)
body_outline_indicator = (1-body_outline_mask) * 255

In [ ]:
def GenerateMask(img):
    mask = numpy.zeros(img.shape)
    contours, _ = cv2.findContours(
        cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_NONE
    )
    contours = sorted(contours, key=cv2.contourArea, reverse= True)
    cv2.drawContours(mask, contours, 0, color=(1,1,1), thickness=cv2.FILLED)
    return mask

In [ ]:
segmentor = SelfiSegmentation()

In [ ]:
cv2.namedWindow("Camera Window")
img_counter = 0
while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        seg_img = segmentor.removeBG(frame, (0,0,0), threshold=0.7)

        frame = cv2.resize(frame,(512,512),cv2.INTER_AREA)
        seg_img = cv2.resize(seg_img,(512,512),cv2.INTER_AREA)

        mask = GenerateMask(seg_img)

        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, numpy.multiply(frame,mask))
        cv2.imwrite("mask_"+img_name, mask * 255)

        print("{} written!".format(img_name))
        img_counter += 1
    else:
        frame = cv2.resize(frame,(512,512), cv2.INTER_AREA)
        frame = numpy.multiply(frame,body_outline_mask) + body_outline_indicator
        cv2.imshow("Camera Window", frame)
cam.release()
cv2.destroyAllWindows()